In [1]:
import json, yaml
import os
from datetime import datetime
from utils.gen_link_info import gen_port_info

In [2]:
# set env vars
os.environ['MLIB_DEVEL_PATH'] = '/home/wei/scilab_demo/mlib_devel'
os.environ['HDL_ROOT'] = '/home/wei/scilab_demo/mlib_devel/scilab_library/hdl_sources'

In [3]:
# write the blkinfo to the dict
def write_blkinfo_to_dict(blk_info, blk_tmp):
    # make a copy of the dict
    template = blk_tmp.copy()
    keys = list(template.keys())
    index = blk_info['id']
    for i in range(len(index)):
        try:
            template[keys[index[i][0]]] = int(blk_info['val'][i][0])
        except:
            try:
                template[keys[index[i][0]]] = float(blk_info['val'][i][0])
            except:
                if blk_info['val'][i][0] == 'on':
                    template[keys[index[i][0]]] = True
                elif blk_info['val'][i][0] == 'off':
                    template[keys[index[i][0]]] = False
                else:
                    template[keys[index[i][0]]] = blk_info['val'][i][0]
    # go through all of the k-v pairs in the template, 
    # and modify all of the 'on' to True and 'off' to False
    for k, v in template.items():
        if v == 'on':
            template[k] = True
        elif v == 'off':
            template[k] = False
    return template

In [4]:
# get the scilab path from the environment variable
mlib_devel_path = os.getenv('MLIB_DEVEL_PATH')
scilab_library_path = mlib_devel_path + '/scilab_library'
# open the jasper.json file
with open('%s/jasper.json'%mlib_devel_path) as f:
    blkinfo = json.load(f)
# after reading the file, remove it
# os.remove('jasper.json')
# open the block_info_template.json file
with open('%s/block_info_template.json'%(scilab_library_path)) as f:
    blkinfo_template = json.load(f)

In [5]:
blkfn = blkinfo['project']['filename']
proj_name = blkfn.split('/')[-1].split('.')[0]
filepath = blkfn.split('.')[0]

In [7]:
# get the scilab blk, blk objs and link objs from the jaser.json file
scilab_blk_objs = []
blk_objs = []
link_objs = []
for k, v in blkinfo.items():
    if k.startswith('blk'):
        if v['tag'].startswith('scilab-blk'):
            scilab_blk_objs.append(v)
        else:
            blk_objs.append(v)
    if k.startswith('link'):
        link_objs.append(v)

# genenrate yellow blocks dict
yellow_blocks = {}
for blk in blk_objs:
    tag = blk['tag']
    # check if the block in the blockinfo starts with xps
    if blkinfo_template[tag]['tag'].startswith('xps'):
        # populate yellow block
        key = proj_name + '/' + blk['val'][0][0]
        yellow_blocks[key] = write_blkinfo_to_dict(blk, blkinfo_template[tag])
        yellow_blocks[key]['fullpath'] = key
user_modules = {}
jasper_per = {}
jasper_per['yellow_blocks'] = yellow_blocks
jasper_per['user_modules'] = user_modules

In [8]:
# now, we need to generate jasper.dsp, which contains the dsp block info
dsp_blocks = {}
for blk in blk_objs:
    tag = blk['tag']
    # check if the block in the blockinfo starts with dsp or xps:xsg
    # we need xps:xsg as it contains the platform info
    if blkinfo_template[tag]['tag'].startswith('dsp') or blkinfo_template[tag]['tag'] == 'xps:xsg':
        # populate dsp block
        key = proj_name + '/' + blk['val'][0][0]
        dsp_blocks[key] = write_blkinfo_to_dict(blk, blkinfo_template[tag])
        dsp_blocks[key]['fullpath'] = key
# generate jasper.dsp
jasper_dsp = {}
jasper_dsp['dsp_blocks'] = dsp_blocks

In [9]:
xps_link_info, dsp_link_info, xps_dsp_link_info = gen_port_info(blkinfo)

In [10]:
xps_link_info

{}

In [11]:
dsp_link_info

{}

In [12]:
xps_dsp_link_info

{'adder': {'in': [1, 2], 'out': [1]}}